In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('sale_table.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   entrp_ptnt_id  37 non-null     int64 
 1   store_nbr      37 non-null     int64 
 2   sale_dt        37 non-null     object
dtypes: int64(2), object(1)
memory usage: 1020.0+ bytes


In [4]:
df.head(37)

,entrp_ptnt_id,store_nbr,sale_dt
0,1004,401,2016-01-01
1,1003,301,2022-03-10
2,1001,103,2021-01-01
3,1006,601,2023-12-01
4,1008,802,2021-07-01
5,1005,502,2023-06-30
6,1001,104,2023-01-01
7,1005,502,2023-07-30
8,1005,502,2023-05-31
9,1003,304,2022-08-07


In [24]:
# Sort data by patient ID (entrp_ptnt_id)
df_sorted = df.sort_values(by=['entrp_ptnt_id', 'sale_dt'], ascending=[True, True])
print(f"Data sorted by 'entrp_ptnt_id'")
print(f"Original data shape: {df.shape}")
print(f"Sorted data shape: {df_sorted.shape}")

print(f"\nFirst 10 rows of sorted data:")
print(df_sorted.head(10))

print(f"\nLast 10 rows of sorted data:")
print(df_sorted.tail(10))

# Update the original dataframe with sorted data
op_df = df_sorted
print(f"\nOriginal dataframe 'df' has been updated with sorted data")

Data sorted by 'entrp_ptnt_id'
Original data shape: (37, 3)
Sorted data shape: (37, 3)

First 10 rows of sorted data:
    entrp_ptnt_id  store_nbr     sale_dt
16           1001        101  2020-01-01
20           1001        101  2020-04-20
23           1001        102  2020-04-21
34           1001        200  2020-04-21
26           1001        101  2020-12-31
2            1001        103  2021-01-01
18           1001        101  2022-12-31
6            1001        104  2023-01-01
10           1002        201  2018-06-01
14           1002        202  2019-07-06

Last 10 rows of sorted data:
    entrp_ptnt_id  store_nbr     sale_dt
21           1005        502  2023-08-29
36           1005        502  2023-09-28
3            1006        601  2023-12-01
29           1007        701  2019-05-05
17           1007        701  2019-08-23
13           1007        702  2020-05-04
27           1007        703  2022-05-04
4            1008        802  2021-07-01
25           1008        801  20

In [25]:
#retrieve data of specific patient
print(op_df[op_df['entrp_ptnt_id'] == 1001])

op_df['sale_dt'] = pd.to_datetime(op_df['sale_dt'])


#new column for first purchase
op_df['first_purchase'] = op_df.groupby('entrp_ptnt_id')['sale_dt'].transform('min')

#new column with days since first purchase
op_df['days_since_first_purchase'] = (op_df['sale_dt'] - op_df['first_purchase']).dt.days

    entrp_ptnt_id  store_nbr     sale_dt
16           1001        101  2020-01-01
20           1001        101  2020-04-20
23           1001        102  2020-04-21
34           1001        200  2020-04-21
26           1001        101  2020-12-31
2            1001        103  2021-01-01
18           1001        101  2022-12-31
6            1001        104  2023-01-01


In [26]:
op_df.head(10)

,entrp_ptnt_id,store_nbr,sale_dt,first_purchase,days_since_first_purchase
16,1001,101,2020-01-01,2020-01-01,0
20,1001,101,2020-04-20,2020-01-01,110
23,1001,102,2020-04-21,2020-01-01,111
34,1001,200,2020-04-21,2020-01-01,111
26,1001,101,2020-12-31,2020-01-01,365
2,1001,103,2021-01-01,2020-01-01,366
18,1001,101,2022-12-31,2020-01-01,1095
6,1001,104,2023-01-01,2020-01-01,1096
10,1002,201,2018-06-01,2018-06-01,0
14,1002,202,2019-07-06,2018-06-01,400


In [ ]:
# Create status column based on days since first purchase
def assign_status(days):
    if days <= 110:
        return 'Active'
    elif days <= 365:
        return 'Inactive'
    elif days <= 1095:
        return 'Lapsed'
    else:
        return 'Lost'

# Apply the status function to each row
op_df['status'] = op_df['days_since_first_purchase'].apply(assign_status)

print("Status column created successfully!")
print("\nStatus distribution:")
print(op_df['status'].value_counts())
print("\nSample data:")
print(op_df[['entrp_ptnt_id', 'sale_dt', 'first_purchase', 'days_since_first_purchase', 'status']].head(10))


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [30]:
op_df.head()

,entrp_ptnt_id,store_nbr,sale_dt,first_purchase,days_since_first_purchase
16,1001,101,2020-01-01,2020-01-01,0
20,1001,101,2020-04-20,2020-01-01,110
23,1001,102,2020-04-21,2020-01-01,111
34,1001,200,2020-04-21,2020-01-01,111
26,1001,101,2020-12-31,2020-01-01,365
